In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib

In [12]:
# Reference:
# https://www.tensorflow.org/tutorials/images/classification
# https://www.tensorflow.org/tutorials/keras/save_and_load

In [2]:
batch_size = 32
img_height = 100
img_width = 100
data_dir = "/Users/patrickbell/Documents/sodukuAI/digitClassifier/trainingImages"

In [3]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  "/Users/patrickbell/Documents/sodukuAI/digitClassifier/trainingImages",
  validation_split=0.2,
  subset="training",
  seed=123,
  batch_size=batch_size,
  image_size=(img_height, img_width))

Found 84960 files belonging to 10 classes.
Using 67968 files for training.


In [4]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 84960 files belonging to 10 classes.
Using 16992 files for validation.


In [5]:
class_names = train_ds.class_names
print(class_names)

['1', '2', '3', '4', '5', '6', '7', '8', '9', 'blank']


In [6]:
# Caches for performance

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [7]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

In [8]:
# Creating the NN

num_classes = 10

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_1 (Rescaling)      (None, 100, 100, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 100, 100, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 50, 50, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0

In [ ]:
epochs=7
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)
model.save('wholeModelSaveWith7Epochs')

Epoch 1/7
2124/2124 [==============================] - 529s 249ms/step - loss: 0.3015 - accuracy: 0.9079 - val_loss: 0.0709 - val_accuracy: 0.9811

In [28]:
# model.save('wholeModelSave')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: wholeModelSave/assets


In [29]:
model.save_weights('savedWeights')